# 進階的 SQL 五十道練習

> 交易控制語言

[數聚點](https://www.datainpoint.com) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
%LOAD mysql db=imdb user=root password='hahowsql'

## （複習）SQL 的分類

- 在初階課程「SQL 的五十道練習」中我們專注在資料查詢語言的部分；這堂進階課程會完整帶學員們認識其他的 SQL 語言。
- 實務上，資料查詢語言是被分類在資料操作語言之下的一個分支。
    - 資料定義語言（Data Definition Language, DDL）
    - 資料操作語言（Data Manipulation Language, DML）
        - 資料查詢語言（Data Query Language, DQL）
    - 資料控制語言（Data Control Language, DCL）
    - **交易控制語言（Transaction Control Language, TCL）**

## 交易控制語言（Transaction Control Language, DCL）主要的保留字

- `COMMIT`
- `ROLLBACK`

這意味著我們可以執行連串的整合型資料操作或者取消連串的整合型資料操作。

## 什麼是交易控制語言

## 不管多麼強韌的系統，我們都無法保證

- 資料庫伺服器「永遠」不會斷線。
- 與資料庫互動的應用程式「永遠」不會中斷、「永遠」不會發生預期之外的錯誤。
- 在進行資料的新增、更新或刪除時，「永遠」不會跟其他的操作同時發生而導致資料鎖定出錯。

## 因此我們需要交易（Transaction）這個機制

- 將多個 SQL 敘述集中執行就稱為交易，若非全數執行成功，要不就是全數失敗未完成（原子性 Atomicity）。
- 確實執行交易後即便伺服器突然關閉，在重新啟動後也會再度套用交易變更內容（持續性 Durability）。
- 透過交易可以避免因操作過程中斷導致資料的不正確。

## 交易的特性

- 原子性（Atomicity）：交易中的操作一定會是「皆未執行」或者「皆已執行」其中一個狀態。
- 持續性（Durability）：交易完成後不會遺失操作結果。
- 一致性（Consistency）：確保資料的完整性。
- 隔離性（Isolation）：執行到一半的操作並不會對其他操作產生影響。

## 以帳戶轉帳為例

- 處理帳戶轉帳的程式必須開始一筆交易（`START TRANSACTION`）。
- 執行轉帳的 SQL 敘述。
- 如果一切順利便會以 `COMMIT` 結束。
- 如果發生了錯誤，就會以 `ROLLBACK` 還原交易開始以來的敘述。

## 虛擬程式碼（Pseudo code）展示帳戶轉帳的例子

```SQL
START TRANSACTION;

UPDATE accounts
   SET balance = balance - 500
 WHERE id = 1;
UPDATE accounts
   SET balance = balance + 500
 WHERE id = 2;

if everything is fine:
    COMMIT;
else:
    ROLLBACK; 
```

## 帳戶轉帳

## 建立一個資料庫 `tcl`

In [2]:
CREATE DATABASE tcl CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;

## 建立一個資料表 `accounts`

In [3]:
CREATE TABLE tcl.accounts (
    id int unsigned AUTO_INCREMENT,
    balance double unsigned,
    PRIMARY KEY (id)
);

In [4]:
INSERT INTO tcl.accounts (balance) VALUES
    (1000),
    (1000);

## 建立一個資料表 `transfers`

In [5]:
CREATE TABLE tcl.transfers (
    id int unsigned AUTO_INCREMENT,
    from_account_id int unsigned,
    to_account_id int unsigned,
    amount double,
    PRIMARY KEY (id)
);

## 提交與回滾

## 交易的開始與結束

- 交易的開始以 `START TRANSACTION` 標註。
- 交易的結束：
    - 一切正常無誤，執行 `COMMIT` 提交。
    - 發生任何錯誤，執行 `ROLLBACK` 回滾。

## 交易的開始：轉帳 500 元

In [7]:
START TRANSACTION;

In [8]:
INSERT INTO tcl.transfers (from_account_id, to_account_id, amount) VALUES
    (1, 2, 500);

In [9]:
UPDATE tcl.accounts
   SET balance = balance + 500
 WHERE id = 2;

In [10]:
UPDATE tcl.accounts
   SET balance = balance - 500
 WHERE id = 1;

## 確認無誤，完成交易

In [11]:
SELECT *
  FROM tcl.accounts;

id,balance
1,500
2,1500


In [12]:
SELECT *
  FROM tcl.transfers;

id,from_account_id,to_account_id,amount
1,1,2,500


In [13]:
COMMIT;

## 再開始另一次交易

In [14]:
START TRANSACTION;

In [15]:
INSERT INTO tcl.transfers (from_account_id, to_account_id, amount) VALUES
    (1, 2, 600);

In [16]:
UPDATE tcl.accounts
   SET balance = balance + 600
 WHERE id = 2;

In [17]:
UPDATE tcl.accounts
   SET balance = balance - 600
 WHERE id = 1;

Error: Out of range value for column 'balance' at row 1 while executing "UPDATE tcl.accounts
   SET balance = balance - 600
 WHERE id = 1;".

## 帳戶餘額不能為負數

- `balance` 資料類型 `unsigned`
- `INSERT INTO tcl.transfers` 與 `UPDATE...WHERE id = 2` 已經執行，導致帳目無法勾稽。

In [18]:
SELECT *
  FROM tcl.accounts;

id,balance
1,500
2,2100


In [19]:
SELECT *
  FROM tcl.transfers;

id,from_account_id,to_account_id,amount
1,1,2,500
2,1,2,600


## 以 `ROLLBACK` 回滾，結束交易

In [20]:
ROLLBACK;

In [21]:
SELECT *
  FROM tcl.accounts;

id,balance
1,500
2,1500


In [22]:
SELECT *
  FROM tcl.transfers;

id,from_account_id,to_account_id,amount
1,1,2,500


## 重點統整

- 將多個 SQL 敘述集中執行就稱為交易（Transaction），若非全數執行成功，要不就是全數失敗未完成。
- 交易的特性
    - 原子性（Atomicity）
    - 持續性（Durability）
    - 一致性（Consistency）
    - 隔離性（Isolation）

## 重點統整（續）

- 交易的開始以 `START TRANSACTION` 標註。
- 交易的結束：
    - 一切正常無誤，執行 `COMMIT` 提交。
    - 發生任何錯誤，執行 `ROLLBACK` 回滾。